# Agentic Workflow Execution 

This notebook is used to execute Agentic Workflows defined in **agentic_workflows.ipynb**

Required Parameters:
- workflow_name: Name of the workflow to be executed
- query: Input query of the workflow

Example SPL:

| makeresults
| fit MLTKContainer algo=agentic_workflow_execution workflow_name=SimpleLLMFlow query="What does Splunk do?" * into app:agentic_workflow_execution

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
import os
import importlib
import asyncio
import concurrent.futures

# Run async agent in sync
def run_async_in_sync(async_func):
    loop = asyncio.get_event_loop()
    if loop.is_running():
        with concurrent.futures.ThreadPoolExecutor() as pool:
            return pool.submit(lambda: asyncio.run(async_func())).result()
    else:
        return asyncio.run(async_func())
# ...
# global constants
module_path = "app.model.agentic_workflows"
MODEL_DIRECTORY = "/srv/app/model/data/"

In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)

numpy version: 1.26.4
pandas version: 2.2.0


## Stage 1 - get a data sample from Splunk
There are currently 2 ways to retrieve data from Splunk: Option 1 is to interactively pull data from Splunk into the DSDL Jupyter Lab environment. This is useful when the Splunk REST API is accessible from the Jupyter environment and a valid Splunk auth token is defined in the DSDL app. This option has advantages to quickly experiment with different Splunk SPL queries and further interactively work with the search results in Jupyter. Note that no meta data param object is retrieved here. This is only available in Stage 1 Option 2.

### Option 1 - pull data from Splunk

In [6]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [7]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("agentic_workflow_execution")

## Stage 2 - create and initialize a model

In [10]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}
    model['hyperparameter'] = 42.0
    return model

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
print(model)

## Stage 3 - fit the model

In [12]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    # model.fit()
    info = {"message": "model trained"}
    return info

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print(fit(model,df,param))

## Stage 4 - apply the model

In [6]:
# apply your model
def apply(model,df,param):
    try:
        query = param['options']['params']['query'].strip('\"')
    except:
        result = pd.DataFrame({'Message': ["ERROR: Please input a parameter \'prompt\'."]})
        return result
        
    try:
        workflow_name = param['options']['params']['workflow_name'].strip('\"')
    except:
        result = pd.DataFrame({'Message': ["ERROR: Please input a parameter \'workflow_name\'."]})
        return result
        
    try:
        llm_utils_module = importlib.import_module(module_path)
        dynamic_function = getattr(llm_utils_module, workflow_name)
        print(f"Successfully imported '{workflow_name}' from '{module_path}'.")
        print(f"Type of dynamic_function: {type(dynamic_function)}")

        w = dynamic_function(timeout=6000, verbose=False)
        async def agent_run():
            # Run the agent
            response = await w.run(query=query)
            return response
        output = run_async_in_sync(agent_run)
        result = pd.DataFrame({'Response': [output]})
        return result      
    
    except ImportError as e:
        error = f"Error importing module '{module_path}': {e}"
        result = pd.DataFrame({'Message': [error]})
        return result
    except AttributeError as e:
        error = f"Error finding attribute '{a}' in module '{module_path}': {e}"
        result = pd.DataFrame({'Message': [error]})
        return result
    except Exception as e:
        error = f"An unexpected error occurred: {e}"
        result = pd.DataFrame({'Message': [error]})
        return result

In [ ]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = None
df = pd.DataFrame({'Message': ["test"]})
param = {'options':{'params': {'query': "What is Splunk", 'workflow_name': "SimpleLLMFlow"}}}
d = apply(model,df,param)
print(d)

## Stage 5 - save the model

In [16]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
        json.dump(model, file)
    return model

## Stage 6 - load the model

In [17]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = {}
    with open(MODEL_DIRECTORY + name + ".json", 'r') as file:
        model = json.load(file)
    return model

## Stage 7 - provide a summary of the model

In [ ]:
# return a model summary
def summary(model=None):
    returns = {"version": {"numpy": np.__version__, "pandas": pd.__version__} }
    return returns

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code